In [1]:
import xml.etree.ElementTree as ET
import csv, os

xml_files = [file for file in os.listdir(os.getcwd()+"/data") if file.endswith(".xml")]
xml_files

['RTE1_dev1_3ways.xml',
 'RTE2_test_3ways.xml',
 'RTE3_test_3ways.xml',
 'RTE3_dev_3ways.xml',
 'RTE1_test_3ways.xml',
 'RTE2_dev_3ways.xml',
 'RTE1_dev2_3ways.xml']

In [2]:
import spacy
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle
ps = PorterStemmer()
nlp = spacy.load('en_core_web_lg')

In [3]:
text = []
hypo = []
label = []
for file in xml_files:
    tree = ET.parse("./data/" + file)
    root = tree.getroot()
    pairs = root.findall('pair')
    for pair in pairs:
        if (pair.attrib)['entailment'] != "UNKNOWN":
            text.append(pair.find('t').text)
            hypo.append(pair.find('h').text)
            label.append((pair.attrib)['entailment'])

In [4]:
import pandas as pd
df = pd.DataFrame({"text" : text,"hypo" : hypo,"label" : label})

In [5]:
df.head(20)


,hypo,label,text
0,Crude oil prices rose to $37.80 per barrel,NO,Crude oil for April delivery traded at $37.80 ...
1,Oracle released a confidential document,NO,Oracle had fought to keep the forms from being...
2,Strong sales for iTunes in Europe.,YES,iTunes software has seen strong sales in Europe.
3,Companies selling genetically modified foods d...,NO,"All genetically modified food, including soya ..."
4,Coffee drinking has health benefits.,YES,Researchers at the Harvard School of Public He...
5,Fiber improves blood sugar control.,YES,Eating lots of foods that are a good source of...
6,Rock band Phish holds final concert in Vermont.,YES,Phish disbands after a final concert in Vermon...
7,Denmark and Sweden tie.,YES,Euro-Scandinavian media cheer Denmark v Sweden...
8,translator kidnapped in Iraq,YES,Iraqi militants said Sunday they would behead ...
9,Green card is now difficult to receive.,YES,Green cards are becoming more difficult to obt...


In [6]:
from keras.preprocessing import text, sequence
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import spacy
nlp = spacy.load('en_core_web_lg')
import fasttext as ft
import pandas as pd
import numpy as np
import re,os

def preprocess_text(text):
        lem_words = []
        doc = nlp(text)
        for token in doc:
            if token.is_alpha == True:
                lem_words.append(token.lemma_)
            else:
                lem_words.append("_value_")
        text = " ".join(lem_words)
        return text

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
df['text'] = df.apply(lambda text: preprocess_text(text[2]), axis=1)
df['hypo'] = df.apply(lambda text: preprocess_text(text[0]), axis=1)


Using TensorFlow backend.


In [7]:
df.head(10)

,hypo,label,text
0,crude oil price rise to _value_ _value_ per ba...,0,crude oil for april delivery trade at _value_ ...
1,oracle release a confidential document,0,oracle have fight to keep the form from be rel...
2,strong sale for itunes in europe _value_,1,itunes software have see strong sale in europe...
3,company sell genetically modify food do _value...,0,all genetically modify food _value_ include so...
4,coffee drinking have health benefit _value_,1,researcher at the harvard school of public hea...
5,fiber improve blood sugar control _value_,1,eat lot of food that be a good source of fiber...
6,rock band phish hold final concert in vermont ...,1,phish disband after a final concert in vermont...
7,denmark and sweden tie _value_,1,euro _value_ scandinavian medium cheer denmark...
8,translator kidnap in iraq,1,iraqi militant say sunday -PRON- would behead ...
9,green card be now difficult to receive _value_,1,green card be become more difficult to obtain ...


In [34]:
def split_training_data(df):
        a, b, c,d = model_selection.train_test_split(df['text'], df['label'],random_state=1234)
        aa, bb, c,d = model_selection.train_test_split(df['hypo'], df['label'],random_state=1234)
        return a , b , aa, bb , c , d
train_t, valid_t, train_h, valid_h, train_y, valid_y = split_training_data(df)

In [23]:
train_t.head(20)

2749    at the southeastern tip of the city _value_ xo...
1298    the bolan pass be a gap through the toba kakar...
1619    mr carstens leave -PRON- job as imf _value_ de...
1141    a specialist firm on the new york stock exchan...
1692    the directive _value_ issue by vietnam _value_...
301     at the same time _value_ old mutual asset mana...
2448    sani _value_ seat can offset the rise cost of ...
2086    de la rue be the world _value_ large commercia...
1023    two document that a former fbi agent say -PRON...
1763    today _value_ highlight in history _value_ on ...
83      passion surround germany _value_ final match a...
2585    india _value_ an enchanting country situate in...
216     speak to newsman in jakarta today _value_ hun ...
1838    mark said and philip dick _value_ two member o...
79      wal _value_ mart have receive a lot of negativ...
2347    the controversy become more desperate _value_ ...
2670    a cataclysmic starquake be think to have cause...
527     recogn

In [24]:
train_h.head(20)

2749    xochimilco be a popular tourist attraction bec...
1298    the british army cross the toba kakar range of...
1619    mr carstens lose -PRON- job as imf _value_ dep...
1141    one judge say that the nearly half million dol...
1692    the regulation be the late one in a string of ...
301      barrow hanley be a competitor of pacific _value_
2448                    the cost of paper be rise _value_
2086    de la rue make much of the world _value_ money...
1023      martin luther king be murder in _value_ _value_
1763    lee harvey oswald kill president john _value_ ...
83         a man be stab because of a soccer game _value_
2585              india be on the asian continent _value_
216             hun sen represent the khmer rouge _value_
1838    mark said work for ontario bradley first air _...
79      wal _value_ mart face allegation for underpay ...
2347    the kuiper belt be nine billion mile from the ...
2670    the flash of radiation on december _value_ _va...
527      lake 

In [25]:
train_y.head(20)

2749    1
1298    1
1619    0
1141    1
1692    1
301     0
2448    1
2086    1
1023    1
1763    1
83      1
2585    1
216     0
1838    1
79      1
2347    1
2670    1
527     0
2704    1
190     0
Name: label, dtype: int64

In [12]:
df["combined"] = df.apply(lambda x : "{} {}".format(x[0] , x[2]),axis=1)
df["combined"]

0       crude oil price rise to _value_ _value_ per ba...
1       oracle release a confidential document oracle ...
2       strong sale for itunes in europe _value_ itune...
3       company sell genetically modify food do _value...
4       coffee drinking have health benefit _value_ re...
5       fiber improve blood sugar control _value_ eat ...
6       rock band phish hold final concert in vermont ...
7       denmark and sweden tie _value_ euro _value_ sc...
8       translator kidnap in iraq iraqi militant say s...
9       green card be now difficult to receive _value_...
10      british serviceman detain iran will soon relea...
11      -PRON- take long to get green card _value_ the...
12      coal stock rise _value_ coal company stock get...
13      british serviceman detain the royal navy servi...
14      coal stock rise _value_ total coal stock with ...
15      california driver _value_ license grant to ill...
16      south korea continue to send troop _value_ sou...
17      clinto

In [15]:
def making_embeddings_and_sequences(train_df,train_t,valid_t,train_h,valid_h):

        embeddings_index = {}
        for i, line in enumerate(open('data/subword.vec'.format(os.getcwd()))):
            values = line.split()
            embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
        token_t = text.Tokenizer()
        token_t.fit_on_texts(train_df['combined'])
        word_index = token_t.word_index
        max_len_t = train_df.text.map(len).max()
        train_seq_t = sequence.pad_sequences(token_t.texts_to_sequences(train_t), maxlen=max_len_t)
        valid_seq_t = sequence.pad_sequences(token_t.texts_to_sequences(valid_t), maxlen=max_len_t)
        train_seq_h = sequence.pad_sequences(token_t.texts_to_sequences(train_h), maxlen=max_len_t)
        valid_seq_h = sequence.pad_sequences(token_t.texts_to_sequences(valid_h), maxlen=max_len_t)
        embedding_matrix = np.zeros((len(word_index) + 1, 300))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        

        return train_seq_t, valid_seq_t, word_index, embedding_matrix,train_seq_h, valid_seq_h
    

In [16]:
train_seq_t, valid_seq_t, word_index, embedding_matrix,train_seq_h, valid_seq_h= making_embeddings_and_sequences(df,train_t, valid_t,train_h, valid_h)

In [29]:
from keras import utils
from sklearn.metrics import confusion_matrix
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Bidirectional
from keras.models import model_from_json
from sklearn import model_selection
import keras
from keras.models import Model
from keras.layers import Input
from keras.optimizers import SGD
import tensorflow as tf
import os

In [30]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

In [38]:
def create_the_model(len_word_index,embedding_matrix,num_classes):
        input_t = Input(shape=(743,), dtype='float32')
        input_h = Input(shape=(743,), dtype='float32')
        t_input = Embedding(len_word_index + 1, 300, weights=[embedding_matrix], trainable=False)(input_t)
        bi_t = Bidirectional(LSTM(units=5))(t_input)
        bi_t = Dropout(0.5)(bi_t)
        h_input = Embedding(len_word_index + 1, 300, weights=[embedding_matrix], trainable=False)(input_h)
        bi_h = Bidirectional(LSTM(units=5))(h_input)
        bi_h = Dropout(0.5)(bi_h)
        conc = keras.layers.concatenate([bi_t, bi_h],axis=-1)
        x = Dense(units=128,activation="relu")(conc)
        x = Dropout(0.5)(x)
        x = Dense(units=64,activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(units=32,activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(units=num_classes,activation="softmax")(x)
        model = Model(inputs=[input_t, input_h], outputs=[x])
        adam = keras.optimizers.Adam(lr=0.0001, decay=0.0000001, amsgrad=False)
        model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy'])
        return model

In [35]:
model = None
model_name = "lstm_combined_text_simple"
num_classes = np.max(train_y) + 1
print(num_classes)
train_y = utils.to_categorical(train_y, num_classes)
print(train_y)
valid_y = utils.to_categorical(valid_y, num_classes)


2
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]
Epoch 1/1
2192/2192 [==============================] - 24s 11ms/step - loss: 0.5675 - acc: 0.7724


In [39]:
model = create_the_model(len(word_index),embedding_matrix,num_classes)
model.fit([train_seq_t,train_seq_h], train_y, epochs=10, batch_size=256)
#validation_data=([self.data_processing.valid_seq_t,self.data_processing.valid_seq_h], self.data_processing.valid_y)
y_predict = model.predict([valid_seq_t,valid_seq_h])
#print(y_predict)

Epoch 1/10
2192/2192 [==============================] - 25s 11ms/step - loss: 0.6911 - acc: 0.5753
Epoch 2/10
2192/2192 [==============================] - 21s 9ms/step - loss: 0.6867 - acc: 0.6756
Epoch 3/10
2192/2192 [==============================] - 21s 10ms/step - loss: 0.6819 - acc: 0.7400
Epoch 4/10
2192/2192 [==============================] - 21s 10ms/step - loss: 0.6761 - acc: 0.7559
Epoch 5/10
2192/2192 [==============================] - 21s 10ms/step - loss: 0.6706 - acc: 0.7678
Epoch 6/10
2192/2192 [==============================] - 21s 9ms/step - loss: 0.6635 - acc: 0.7719
Epoch 7/10
2192/2192 [==============================] - 21s 9ms/step - loss: 0.6545 - acc: 0.7737
Epoch 8/10
2192/2192 [==============================] - 21s 10ms/step - loss: 0.6447 - acc: 0.7765
Epoch 9/10
2192/2192 [==============================] - 21s 10ms/step - loss: 0.6343 - acc: 0.7787
Epoch 10/10
2192/2192 [==============================] - 22s 10ms/step - loss: 0.6230 - acc: 0.7769


In [36]:
print(y_predict)

[[0.26834384 0.7316562 ]
 [0.2664134  0.73358667]
 [0.2602133  0.7397868 ]
 ...
 [0.2646246  0.7353754 ]
 [0.26240775 0.7375923 ]
 [0.2626068  0.73739314]]
